In [1]:
!pip install transformers

In [14]:
import pandas as pd
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [15]:
data = pd.read_csv('/content/LLM-Sample-Input-File.csv')

In [16]:
data['Value - Randomized'] = data['Value - Randomized'].apply(lambda x: f'{float(x):.2f}')
data.head(20)

,Company Name,Category,Sub Cat,Period,Value - Randomized
0,Potato Inc.,Revenue By Region,Americas,2015 Q1,183000000000.00
1,Potato Inc.,Revenue By Region,Americas,2015 Q2,171000000000.00
2,Potato Inc.,Revenue By Region,Americas,2015 Q3,80840000000.00
3,Potato Inc.,Revenue By Region,Americas,2015 Q4,65310000000.00
4,Potato Inc.,Revenue By Region,Americas,2016 Q1,235000000000.00
5,Potato Inc.,Revenue By Region,Americas,2016 Q2,153000000000.00
6,Potato Inc.,Revenue By Region,Americas,2016 Q3,35920000000.00
7,Potato Inc.,Revenue By Region,Americas,2016 Q4,121000000000.00
8,Potato Inc.,Revenue By Region,Americas,2017 Q1,128000000000.00
9,Potato Inc.,Revenue By Region,Americas,2017 Q2,190000000000.00


In [17]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [19]:
def generate_answer(user_question, model):
    input_text = "Question: " + user_question + " Answer:"
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

    output = model.generate(input_ids, max_length=100, num_return_sequences=1, no_repeat_ngram_size=2, early_stopping=True)

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    answer_start = generated_text.find("Answer:") + len("Answer:")
    answer = generated_text[answer_start:].strip()
    return answer

In [21]:
while True:
    user_query = input("Enter your question (or 'exit' to quit): ")
    if user_query.lower() == 'exit':
        break
    else:
        answer = generate_answer(user_query, model)
        print("Answer:", answer)

Enter your question (or 'exit' to quit): “How much revenue does Potato Inc. make from selling Smartphones


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Answer: $1.5 million.

Question 2: What is the average revenue per user for Smartphone sales? Answer
...
 (1) The average user of a SmartPhone is $2.50 per month. (2)
, (3) Smart phones are sold in the United States, Canada, Australia, New Zealand, and the European Union. The United Kingdom is one of
Enter your question (or 'exit' to quit): “How much revenue does Potato Inc. make from Japan”


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Answer: $1.5 billion.

Source: http://www.kotaku.com/articles/2014/11/14/potato-inc.html
.
Enter your question (or 'exit' to quit): exit
